In [1]:
from can.interface import Bus
from cantools.database import load_file
from broqer import Sink
from canbro import Node
import ipywidgets as widgets

ldf is not supported
xls is not supported
xlsx is not supported


In [2]:
# load dbc file
db = load_file('poc/device_CAN.dbc')

In [3]:
# create ECU node with virtual bus test
bus_e= Bus('test', interface='virtual')
ecu = Node(name="ECU",bus=bus_e,database=db )

In [4]:
# create VCU node with virtual bus test, and connect to ECU via same name of bus -> test
bus_v= Bus('test', interface='virtual')
vcu= Node(name='CONTROL',bus=bus_v,database=db )

In [7]:
vcu.DEM._signal_operation_mode._metadata.initial=0

In [8]:
vcu.DEM.start_periodic()

In [18]:
vcu.DEM._signal_operation_mode.notify(0)

In [87]:
label_value = widgets.HTML(value="value=?")
def show_vcu_value(value, label: widgets.Label):
    label.value = 'operation mode for ECU node: <br> value={}'.format(value)

show_print = ecu.DEM._signal_operation_mode.subscribe(Sink(show_vcu_value, label=label_value))

In [91]:
list_op_mode = list(set([str(x) for x in list(reversed(vcu.DEM._signal_operation_mode._metadata.conversion.choices.values()))]))
vcu_mode = widgets.ToggleButtons(
    options=list_op_mode,
    description=ecu.DEM._signal_operation_mode._metadata.name + ' selection for VCU node:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=list_op_mode,
    value = str(vcu.DEM._signal_operation_mode._state),
#     icons=['check'] * 3
)
def set_vcu_mode(value):
    #print('set value to {}'.format(value))
    vcu.DEM._signal_operation_mode.notify(value['new'])
vcu_mode.observe(set_vcu_mode, 'value')

In [88]:
def val_vcu_value(value):
    global a_op_mode
    a_op_mode = value

show_print_var = ecu.DEM._signal_operation_mode.subscribe(Sink(val_vcu_value))

In [89]:
display(vcu_mode)
display(label_value)

ToggleButtons(description='operation_mode selection for VCU node:', options=('unused', 'Standby', 'Boost', 'Bu…

HTML(value='operation mode for ECU node: <br> value=Boost')